### Initialize Data

In [1]:
import torch
import torch.optim as optim
from focal_loss import FocalLoss
from mock_dataset import MockOutfitDataset
from outfit_model import OutfitCompatibilityModel
import torch.nn as nn
from utils import save_checkpoint
import logging

logging.basicConfig(level=logging.DEBUG)

# Instantiate the mock dataset and dataloader
# Contains list of all outfits
mock_dataset = MockOutfitDataset()


# Organizes your dataset into batches.
# Batch size = number of samples processed in one iteration
# Number of batches = total samples divided by batch_size
# Each this case, a sample = an outfit
def custom_collate(batch):
    images = torch.stack([item["outfit_images"] for item in batch], dim=0)
    texts = [item["outfit_texts"] for item in batch]
    labels = torch.stack([item["outfit_labels"] for item in batch], dim=0)

    return images, texts, labels


dataloader = torch.utils.data.DataLoader(
    mock_dataset, batch_size=15, shuffle=True, collate_fn=custom_collate
)

MockOutfitDataset; images: torch.Size([20, 5, 3, 224, 224]), texts: [['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock descri

### Visualize Data

In [2]:
# CURRENTLY NOT USABLE

# from matplotlib import pyplot as plt
# import torchvision.transforms.functional as F


# def show_images(images, labels):
#     for i in range(images.size(1)):
#         image = F.to_pil_image(images[:, i, ...])
#         plt.subplot(1, images.size(1), i + 1)
#         plt.imshow(image)
#         plt.title(f"Label: {labels[i]}")
#         plt.axis("off")
#     plt.show()


# for batch_idx, (images, texts, labels) in enumerate(dataloader):
#     print(
#         f"Batch {batch_idx + 1} - Shape of images: {images.shape}, Texts: {texts}, Labels: {labels}"
#     )

#     # Visualize the images
#     show_images(images, labels)
    
#     if batch_idx == 2:  # Print information for the first 3 batches
#         break

### Init Model

In [3]:
# Instantiate the model, dataset, and dataloader
model = OutfitCompatibilityModel()
focal_loss = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

c:\Users\phamm\.conda\envs\outfit_recommendation\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\phamm\.conda\envs\outfit_recommendation\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/bert-base-nli-mean-tokens/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transf

### Training

In [4]:
# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    for batch in dataloader:
        images, texts, labels = batch
        # images = batch["outfit_images"]
        # texts = batch["outfit_texts"]
        # labels = batch["outfit_labels"]

        print(f"batch - images.shape: {images.shape}")
        print(f"batch - texts: {texts}")
        print(f"batch - labels: {labels}")

        optimizer.zero_grad()
        outputs = model(images, texts)
        loss = focal_loss(
            outputs, labels.unsqueeze(1)
        )  # Ensure labels have the right dimension
        loss.backward()
        optimizer.step()

        # Print or log the loss if needed
        print(f"Epoch {epoch + 1}, Batch loss: {loss.item()}")

    save_checkpoint(model.state_dict(), "mock", f"model_epoch_{epoch + 1}.pth")

    # Adjust the learning rate as needed (reduce by half in steps of 10)
    if (epoch + 1) % 10 == 0:
        for param_group in optimizer.param_groups:
            param_group["lr"] = param_group["lr"] / 2

DEBUG:root:!!!!!!!!!!
DEBUG:root:OutfitCompatibilityModel - START
DEBUG:root:OutfitCompatibilityModel - intial images.shape: torch.Size([15, 5, 3, 224, 224])
DEBUG:root:@@@@@@@@@@
DEBUG:root:[START LOOP] OUTFIT - 0
DEBUG:root:OutfitCompatibilityModel - outfit_images.shape: torch.Size([5, 3, 224, 224])
DEBUG:root:##########
DEBUG:root:[START LOOP] ITEM - 0
DEBUG:root:OutfitCompatibilityModel - item_index: 0 - item_image.shape: torch.Size([1, 3, 224, 224]) item_text.shape: mock description 0


batch - images.shape: torch.Size([15, 5, 3, 224, 224])
batch - texts: [['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock description 1', 'mock description 2', 'mock description 3', 'mock description 4'], ['mock description 0', 'mock desc

DEBUG:root:ImageEncoder - after fc_layer x's shape: torch.Size([1, 64])
DEBUG:root:----------


----------


DEBUG:root:OutfitCompatibilityModel - item_features.shape: torch.Size([1, 128])
DEBUG:root:[END LOOP] ITEM - 0
DEBUG:root:##########
DEBUG:root:##########
DEBUG:root:[START LOOP] ITEM - 1
DEBUG:root:OutfitCompatibilityModel - item_index: 1 - item_image.shape: torch.Size([1, 3, 224, 224]) item_text.shape: mock description 1
DEBUG:root:ImageEncoder - after fc_layer x's shape: torch.Size([1, 64])
DEBUG:root:----------


...

DEBUG:root:ImageEncoder - after fc_layer x's shape: torch.Size([1, 64])
DEBUG:root:----------
DEBUG:root:OutfitCompatibilityModel - item_features.shape: torch.Size([1, 128])
DEBUG:root:[END LOOP] ITEM - 2
DEBUG:root:##########
DEBUG:root:##########
DEBUG:root:[START LOOP] ITEM - 3
DEBUG:root:OutfitCompatibilityModel - item_index: 3 - item_image.shape: torch.Size([1, 3, 224, 224]) item_text.shape: mock description 3
DEBUG:root:ImageEncoder - after fc_layer x's shape: torch.Size([1, 64])
DEBUG:root:----------


----------
TextEncoder - x's shape after fc_layer: torch.Size([1, 64])
----------
----------


DEBUG:root:OutfitCompatibilityModel - item_features.shape: torch.Size([1, 128])
DEBUG:root:[END LOOP] ITEM - 3
DEBUG:root:##########
DEBUG:root:##########
DEBUG:root:[START LOOP] ITEM - 4
DEBUG:root:OutfitCompatibilityModel - item_index: 4 - item_image.shape: torch.Size([1, 3, 224, 224]) item_text.shape: mock description 4
DEBUG:root:ImageEncoder - after fc_layer x's shape: torch.Size([1, 64])
DEBUG:root:----------
DEBUG:root:OutfitCompatibilityModel - item_features.shape: torch.Size([1, 128])


TextEncoder - x's shape after fc_layer: torch.Size([1, 64])
----------
----------
TextEncoder - x's shape after fc_layer: torch.Size([1, 64])
----------


DEBUG:root:[END LOOP] ITEM - 4
DEBUG:root:##########
DEBUG:root:OutfitCompatibilityModel - curren outfit's feature vectors dimensions: 1 with lengths: [5]
DEBUG:root:OutfitCompatibilityModel - CUR OUTFIT's feature vectors after stack: torch.Size([1, 5, 128])
DEBUG:root:[END LOOP] OUTFIT - 14
DEBUG:root:@@@@@@@@@@
DEBUG:root:OutfitCompatibilityModel - ALL OUTFITS' feature vetors: torch.Size([15, 5, 128])
DEBUG:root:OutfitCompatibilityModel - outfit_token.shape - init: torch.Size([1, 1, 128])
DEBUG:root:OutfitCompatibilityModel - outfit_token.shape - after expand: torch.Size([1, 1, 128])
DEBUG:root:OutfitCompatibilityModel - outfit_features.shape: torch.Size([15, 6, 128])
DEBUG:root:OutfitCompatibilityModel - transformer_output.shape: torch.Size([15, 6, 128])
DEBUG:root:OutfitCompatibilityModel - global_outfit_representation: tensor([[-0.8220,  1.1977,  0.5370,  ..., -0.7296, -0.0055, -0.5198],
        [-1.0402,  1.1723,  0.6157,  ..., -0.9655,  0.1245, -0.6516],
        [-0.8807,  1.316